In [1]:
import re
import time
import pickle
import numpy as np

from edit_distance import SequenceMatcher
import torch
from dataset import SpeechDataset
import matplotlib.pyplot as plt

from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.model import GRUDecoder
import pickle
import argparse
import matplotlib.pyplot as plt
from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.lm_utils import build_llama_1B
from neural_decoder.model import GRUDecoder
from neural_decoder.bit import BiT_Phoneme
import pickle
import argparse
from lm_utils import _cer_and_wer
import json
import os
import copy

/home3/ebrahim/miniconda3/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_sentence(s):
    s = s.lower()
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 "'", ' ']
    ans = []
    for i in s:
        if(i in charMarks):
            ans.append(i)
    
    return ''.join(ans)

In [3]:
base_dir = "/home3/skaasyap/willett"

load_lm = True
# LM decoding hyperparameters
acoustic_scale = 0.8
blank_penalty = np.log(2)

run_for_llm = False

if run_for_llm:
    return_n_best = True
    rescore = False
    nbest = 100
    print("RUNNING IN LLM MODE")
else:
    return_n_best = False
    rescore = False
    nbest = 1
    print("RUNNING IN N-GRAM MODE")

RUNNING IN N-GRAM MODE


In [4]:
if load_lm and 'ngramDecoder' not in globals(): 
        
    lmDir = base_dir +'/lm/languageModel'
    ngramDecoder = lmDecoderUtils.build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I0723 17:07:10.783617 1813061 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0723 17:10:09.470276 1813061 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


In [7]:
# file path stuff for when running models on two servers (obiwan or leia)
# model storage locations and data path within model args differ depending on 
# the server. 
current_server = 'leia'
server_models_ran_on = 'obi'

if current_server == 'obi':
    
    base_path = '/data/willett_data/'
    
    if server_models_ran_on == 'obi':
        model_storage_path = '/data/willett_data/outputs/'
        
    elif server_models_ran_on == 'leia':
        model_storage_path = '/data/willett_data/leia_outputs/'

if current_server == 'leia':
    
    base_path = '/home3/ebrahim/save_transcripts/'
    
    if server_models_ran_on == 'obi':
        model_storage_path = '/home3/ebrahim/obi_outputs/'
    
    elif server_models_ran_on == 'leia':
        model_storage_path = '/home3/skaasyap/willett/outputs'
        
leia_to_obi_dict = {'/home3/skaasyap/willett/data_log_both':'/data/willett_data/ptDecoder_ctc_both', 
                    '/home3/skaasyap/willett/data':'/data/willett_data/ptDecoder_ctc',
                    '/home3/skaasyap/willett/data_log_both_held_out_days': '/data/willett_data/ptDecoder_ctc_both_held_out_days', 
                    '/home3/skaasyap/willett/data_log_both_held_out_days_1': '/data/willett_data/ptDecoder_ctc_both_held_out_days_1',
                    '/home3/skaasyap/willett/data_log_both_held_out_days_2': '/data/willett_data/ptDecoder_ctc_both_held_out_days_2'}

obi_to_leia_dict = {v:k for k, v in leia_to_obi_dict.items()}


In [8]:
device = "cuda:2"

models_to_run = ['gru_linderman_lab_updated']

shared_output_file = ''

if len(shared_output_file) > 0:
    print("Writing to shared output file")
    write_mode = "a"
else:
    write_mode = "w"
    
seeds_list = [0,1,2,3]
partition = "competition" # "test"
run_lm = True

comp_on_reduced = False # if running restricted days analyses
fill_max_day = False
skip_days = [[]]

day_edit_distance = 0
day_seq_length = 0
prev_day = None

if partition == 'test':
    saveFolder_transcripts = f"{base_path}model_transcriptions/"
else:
    saveFolder_transcripts = f"{base_path}model_transcriptions_retest/"
    
day_cer_dict = {}
total_wer_dict = {}

for seed in seeds_list:
    
    day_cer_dict[seed] = []
    total_wer_dict[seed] = []
            
    for mn, model_name_str in enumerate(models_to_run):
        
        effective_length_arr = []
     
        modelPath = f"{model_storage_path}{model_name_str}_seed_{seed}"
        
        if len(shared_output_file) > 0:
            output_file = f"{shared_output_file}_seed_{seed}"
            print(output_file)
        else:
            output_file = f"{model_name_str}_seed_{seed}"
            
        print(f"Running model: {model_name_str}_seed_{seed}")
            
        with open(modelPath + "/args", "rb") as handle:
            args = pickle.load(handle)
            
        # modify data file path based on which server model was ran on 
        # and which server is currently being used to run this notebok.
        if server_models_ran_on != current_server:
            
            if current_server == 'leia':
                data_file = obi_to_leia_dict[args['datasetPath']]
                
            elif current_server == 'obi':
                data_file = leia_to_obi_dict[args['datasetPath']]
                
        
        else:
            
            data_file = args['datasetPath']
            
      
        if 'ventral_6v_only' not in args:
            args['ventral_6v_only'] = False
            
        print(args['restricted_days'])
        trainLoaders, testLoaders, loadedData = getDatasetLoaders(
            data_file, 8, args['restricted_days'], 
            args['ventral_6v_only']
        )
        
        # if true, model is a GRU
        if 'nInputFeatures' in args.keys():
            
            if 'max_mask_pct' not in args:
                args['max_mask_pct'] = 0
            if 'num_masks' not in args:
                args['num_masks'] = 0
            if 'input_dropout' not in args:
                args['input_dropout'] = 0
                
            print("Loading GRU")
            model = GRUDecoder(
                neural_dim=args["nInputFeatures"],
                n_classes=args["nClasses"],
                hidden_dim=args["nUnits"],
                layer_dim=args["nLayers"],
                nDays=args['nDays'],
                dropout=args["dropout"],
                device=device,
                strideLen=args["strideLen"],
                kernelLen=args["kernelLen"],
                gaussianSmoothWidth=args["gaussianSmoothWidth"],
                bidirectional=args["bidirectional"],
                input_dropout=args['input_dropout'], 
                max_mask_pct=args['max_mask_pct'],
                num_masks=args['num_masks'], 
                linderman_lab=args['linderman_lab']
            ).to(device)

        else:
            
            if 'mask_token_zero' not in args:
                args['mask_token_zero'] = False
                            
            # Instantiate model
            # set training relevant parameters for MEMO, doesn't matter for other runs because they are 
            # only run in eval mode.
            model = BiT_Phoneme(
                patch_size=args['patch_size'],
                dim=args['dim'],
                dim_head=args['dim_head'],
                nClasses=args['nClasses'],
                depth=args['depth'],
                heads=args['heads'],
                mlp_dim_ratio=args['mlp_dim_ratio'],
                dropout=0,
                input_dropout=0,
                look_ahead=args['look_ahead'],
                gaussianSmoothWidth=args['gaussianSmoothWidth'],
                T5_style_pos=args['T5_style_pos'],
                max_mask_pct=0.05,
                num_masks=20, 
                mask_token_zeros=args['mask_token_zero'], 
                num_masks_channels=0, 
                max_mask_channels=0, 
                dist_dict_path=0, 
                linderman_lab=False
            ).to(device)
            
            
        ckpt_path = modelPath + '/modelWeights'
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
        model = model.to(device)
        
        model.eval()

        model_outputs = {
            "logits": [],
            "logitLengths": [],
            "trueSeqs": [],
            "transcriptions": [],
        }
        
        total_edit_distance = 0
        total_seq_length = 0

        if partition == "competition":
        
            testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
                
        elif partition == "test":
            
            testDayIdxs = range(len(loadedData[partition])) 
            
        ground_truth_sentences = []
        
        #print("RESTRICTED DAYS: ", args['restricted_days'])
        
        for i, testDayIdx in enumerate(testDayIdxs):
            
            
            day_outputs = {
                "logits": [],
                "logitLengths": [],
                "trueSeqs": [],
                "transcriptions": [],
            }
            
            #if len(skip_days[mn]) > 0:
            #    if testDayIdx in skip_days[mn]:
            #        print("SKIPPING DAY: ", testDayIdx)
            #        continue
            
            if len(args['restricted_days']) > 0:
                if testDayIdx not in args['restricted_days']:
                    continue
                
            test_ds = SpeechDataset([loadedData[partition][i]])
            
            test_loader = torch.utils.data.DataLoader(
                test_ds, batch_size=1, shuffle=False, num_workers=0
            )
            
            for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
                        
                X, y, X_len, y_len, dayIdx = (
                    X.to(device),
                    y.to(device),
                    X_len.to(device),
                    y_len.to(device),
                    torch.tensor([testDayIdx], dtype=torch.int64).to(device),
                )
                
                if args['ventral_6v_only']:
                    X = X[:, :, :128]
                
                if fill_max_day:
                    dayIdx.fill_(args['maxDay'])
                      
                with torch.no_grad():
                    
                    pred = model.forward(X, X_len, dayIdx)
                
                if hasattr(model, 'compute_length'):
                    adjustedLens = model.compute_length(X_len)
                else:
                    adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)
                    
                for iterIdx in range(pred.shape[0]):
                    
                    trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())
                    model_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
                    
                    model_outputs["logitLengths"].append(
                        adjustedLens[iterIdx].cpu().detach().item()
                    )
                    
                    model_outputs["trueSeqs"].append(trueSeq)
                    
                    decodedSeq = torch.argmax(
                        torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                        dim=-1,
                    ) 
                    
                    decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
                    decodedSeq = decodedSeq.cpu().detach().numpy()
                    decodedSeq = np.array([i for i in decodedSeq if i != 0])
                    
                    matcher = SequenceMatcher(
                        a=trueSeq.tolist(), b=decodedSeq.tolist()
                    )
                    
                    total_edit_distance += matcher.distance()
                    total_seq_length += len(trueSeq)
                    
                    day_edit_distance += matcher.distance()
                    day_seq_length += len(trueSeq)
                    
                transcript = loadedData[partition][i]["transcriptions"][j].strip()
                transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
                transcript = transcript.replace("--", "").lower()
                model_outputs["transcriptions"].append(transcript)
                
            cer_day = day_edit_distance / day_seq_length
            day_cer_dict[seed].append(cer_day)
            print("CER DAY: ", cer_day)
            day_edit_distance = 0 
            day_seq_length = 0

        cer = total_edit_distance / total_seq_length
        
        print("Model performance: ", cer)

        if run_lm:
            
            print("Running n-gram LM")
            
            llm_outputs = []
            start_t = time.time()
            nbest_outputs = []
            
            sample_times = []

            for j in range(len(model_outputs["logits"])):
                logits = model_outputs["logits"][j]

                # Move blank token to the end
                logits = np.concatenate([logits[:, 1:], logits[:, 0:1]], axis=-1)

                # Rearrange logits for decoding
                logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)

                # Time individual sample decoding
                sample_start = time.time()
                nbest = lmDecoderUtils.lm_decode(
                    ngramDecoder,
                    logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
                sample_end = time.time()

                sample_times.append(sample_end - sample_start)
                nbest_outputs.append(nbest)

            # Convert to numpy array for stats
            sample_times = np.array(sample_times)
            mean_time = np.mean(sample_times)
            std_time = np.std(sample_times)

            print(f"Average decoding time per sample: {mean_time:.3f} seconds")
            print(f"Standard deviation: {std_time:.3f} seconds")
 
            if run_for_llm:
                
                print("SAVING OUTPUTS FOR LLM")
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                    pickle.dump(model_outputs, f)
                    
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_nbest.pkl", "wb") as f:
                    pickle.dump(nbest_outputs, f)
                
            else:
                
                # just get perf with greedy decoding
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = model_outputs["transcriptions"][i].strip()
                    nbest_outputs[i] = nbest_outputs[i].strip()
                
                # lower case + remove puncs
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = convert_sentence(model_outputs["transcriptions"][i])

                cer, wer = _cer_and_wer(nbest_outputs, model_outputs["transcriptions"], 
                                    outputType='speech', returnCI=True)

                print("CER and WER after 3-gram LM: ", cer, wer)       
                
                out_file = os.path.join(saveFolder_transcripts, output_file)   # no extension per your spec
                
                with open(out_file + '.txt', write_mode, encoding="utf-8") as f:
                    f.write("\n".join(nbest_outputs)+ "\n")   # one line per LLM output  
                    
                total_wer_dict[seed] = wer
                
#if memo:
#    with open(f"/data/willett_data/paper_results/{model_name_str}_per_memo.pkl", "wb") as f:
#        pickle.dump(day_cer_dict, f)
        
#    with open(f"/data/willett_data/paper_results/{model_name_str}_wer_memo.pkl", "wb") as f:
#        pickle.dump(total_wer_dict, f)ls
        
#else:
#    
#    with open(f"/data/willett_data/paper_results/{model_name_str}_per.pkl", "wb") as f:
#        pickle.dump(day_cer_dict, f)
#        
#    with open(f"/data/willett_data/paper_results/{model_name_str}_wer.pkl", "wb") as f:
#        pickle.dump(total_wer_dict, f)


Running model: gru_linderman_lab_updated_seed_0


KeyError: '/data/willett_data/ptDecoder_ctc'

In [29]:
# TM TRANSFORMER 
0.016 +/- 0.006 seconds


15

In [36]:
0.059/0.016


3.6874999999999996

In [26]:
args['restricted_days']

[4, 5, 6, 7, 8]

In [15]:
args['ventral_6v_only']

True